In [3]:
import nltk
import json
import os
import gzip
import shutil
import fnmatch
import sklearn

from zipfile import ZipFile

from six import string_types

from nltk.tokenize import TweetTokenizer

from nltk.corpus.reader.util import StreamBackedCorpusView, concat, ZipFilePathPointer

from nltk.corpus.reader.api import CorpusReader

from nltk.corpus.reader.twitter import TwitterCorpusReader

from nltk.data import gzip_open_unicode

from sklearn.model_selection import KFold

#Documentation for nltk's twitter corpus reader
#https://www.nltk.org/_modules/nltk/corpus/reader/twitter.html

In [59]:
#These functions exist to unzip gzip files and zip them into regular .zip 
#files, s.t. nltk's TwitterCorpusReader can interpret
def gunzip(file_path,output_path):
    with gzip.open(file_path,"rb") as f_in, open(output_path,"wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
    os.remove(output_path + ".gz")

def recurse_and_gunzip(root):
    walker = os.walk(root)
    for root,dirs,files in walker:
        print(files)
        for f in files:
            if fnmatch.fnmatch(f,"*.gz"):
                gunzip('Twitter-Data/Data/{}'.format(f),'Twitter-Data/Data/{}'.format(f.replace(".gz","")))

def recurse_and_zip(root):
    walker = os.walk(root)
    for root, dirs, files in walker:
        for f in files:
            if fnmatch.fnmatch(f, "*.json"):
                with ZipFile('{}.zip'.format(f), mode = "w") as zf:
                    zf.write('Twitter-Data/Data/{}'.format(f))
                os.remove(f)

# recurse_and_gunzip(root)
# recurse_and_zip(root)

In [114]:
DOC_PATTERN = r'\bManhattan\b'
# CAT_PATTERN = r'^(19|20)\d\d[- /.](0[1-9]|[012])[-/.](0[1-9]|[12][0-9]|3[01])$'
corpus = TwitterCorpusReader('Twitter-Data/Data', "demo4.json")

In [116]:
# corpus.docs()[4]['extended_tweet']
print(type(corpus.docs()))
corpus.docs()[4]['extended_tweet']

<class 'nltk.corpus.reader.util.StreamBackedCorpusView'>


{'extended_entities': {'media': [{'display_url': 'pic.twitter.com/z2vX7SF9OT',
    'indices': [223, 246],
    'sizes': {'small': {'w': 680, 'h': 397, 'resize': 'fit'},
     'large': {'w': 1241, 'h': 725, 'resize': 'fit'},
     'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'medium': {'w': 1200, 'h': 701, 'resize': 'fit'}},
    'id_str': '1192385043689345025',
    'expanded_url': 'https://twitter.com/LawFirmBusiness/status/1192385046801502208/photo/1',
    'media_url_https': 'https://pbs.twimg.com/media/EIwz4TSXkAEA7b0.jpg',
    'id': 1192385043689345025,
    'type': 'photo',
    'media_url': 'http://pbs.twimg.com/media/EIwz4TSXkAEA7b0.jpg',
    'url': 'https://t.co/z2vX7SF9OT'}]},
 'entities': {'urls': [],
  'hashtags': [{'indices': [200, 209], 'text': 'SLFS2019'},
   {'indices': [210, 222], 'text': 'ProfitFirst'}],
  'media': [{'display_url': 'pic.twitter.com/z2vX7SF9OT',
    'indices': [223, 246],
    'sizes': {'small': {'w': 680, 'h': 397, 'resize': 'fit'},
     'large': {'w'

In [60]:
from six import string_types, text_type

from nltk.tokenize import wordpunct_tokenize
from nltk.internals import slice_bounds
from nltk.data import PathPointer, FileSystemPathPointer, ZipFilePathPointer
from nltk.data import SeekableUnicodeStreamReader
from nltk.util import AbstractLazySequence, LazySubsequence, LazyConcatenation, py25
import struct
import io

class GzipStreamBackedCorpusView(StreamBackedCorpusView):
    
    def __init__(self, fileid, block_reader = None, startpos = 0, encoding = 'utf8'):
        StreamBackedCorpusView.__init__(self, fileid, block_reader=block_reader, startpos=0, encoding='utf8')
        
        try:
            if isinstance(self._fileid, PathPointer):
                
                self._eofpos = self.getuncompressedsize(self._fileid)
            else:
                self._eofpos = os.stat(self._fileid).st_size
        except Exception as exc:
            raise ValueError('Unable to open or access %r -- %s' % (fileid, exc))

    def _open(self):
        if isinstance(self._fileid, PathPointer):
#             self._stream = self._fileid.open(self._encoding)
            self._stream = gzip.open(str(self._fileid), 'rb')
#             print(self._stream)
        elif self._encoding:
            print("here1")
            self._stream = SeekableUnicodeStreamReader(
                gzip.open(self._fileid, 'rb'), self._encoding
            )
        else:
            print("here2")
            self._stream = open(self._fileid, 'rb')
    
    def getuncompressedsize(self, filename):
        with gzip.GzipFile(filename, 'r') as fin:
            fin.seek(-4,2)
#             val = struct.unpack('I', fin.read(4))[0]
            val = fin.seek(0, io.SEEK_END)
            print(val)
            return val
    
    def iterate_from(self, start_tok):
        # Start by feeding from the cache, if possible.
        if self._cache[0] <= start_tok < self._cache[1]:
            for tok in self._cache[2][start_tok - self._cache[0] :]:
                yield tok
                start_tok += 1

        # Decide where in the file we should start.  If `start` is in
        # our mapping, then we can jump straight to the correct block;
        # otherwise, start at the last block we've processed.
        if start_tok < self._toknum[-1]:
            block_index = bisect.bisect_right(self._toknum, start_tok) - 1
            toknum = self._toknum[block_index]
            filepos = self._filepos[block_index]
        else:
            block_index = len(self._toknum) - 1
            toknum = self._toknum[-1]
            filepos = self._filepos[-1]

        # Open the stream, if it's not open already.
        if self._stream is None:
            self._open()

        # If the file is empty, the while loop will never run.
        # This *seems* to be all the state we need to set:
        if self._eofpos == 0:
            self._len = 0

        # Each iteration through this loop, we read a single block
        # from the stream.
        while filepos < self._eofpos:
            # Read the next block.
            self._stream.seek(filepos)
            self._current_toknum = toknum
            self._current_blocknum = block_index
            tokens = self.read_block(self._stream)
            assert isinstance(tokens, (tuple, list, AbstractLazySequence)), (
                'block reader %s() should return list or tuple.'
                % self.read_block.__name__
            )
            num_toks = len(tokens)
            new_filepos = self._stream.tell()
            if new_filepos <= filepos:
                print(new_filepos, filepos)
            assert new_filepos > filepos, (
                'block reader %s() should consume at least 1 byte (filepos=%d)'
                % (self.read_block.__name__, filepos)
            )

            # Update our cache.
            self._cache = (toknum, toknum + num_toks, list(tokens))

            # Update our mapping.
            assert toknum <= self._toknum[-1]
            if num_toks > 0:
                block_index += 1
                if toknum == self._toknum[-1]:
                    assert new_filepos > self._filepos[-1]  # monotonic!
                    self._filepos.append(new_filepos)
                    self._toknum.append(toknum + num_toks)
                else:
                    # Check for consistency:
                    assert (
                        new_filepos == self._filepos[block_index]
                    ), 'inconsistent block reader (num chars read)'
                    assert (
                        toknum + num_toks == self._toknum[block_index]
                    ), 'inconsistent block reader (num tokens returned)'

            # If we reached the end of the file, then update self._len
            if new_filepos == self._eofpos:
                self._len = toknum + num_toks
            # Generate the tokens in this block (but skip any tokens
            # before start_tok).  Note that between yields, our state
            # may be modified.
            for tok in tokens[max(0, start_tok - toknum) :]:
                yield tok
            # If we're at the end of the file, then we're done.
            assert new_filepos <= self._eofpos
            if new_filepos == self._eofpos:
                break
            # Update our indices
            toknum += num_toks
            filepos = new_filepos

        # If we reach this point, then we should know our length.
        assert self._len is not None
        # Enforce closing of stream once we reached end of file
        # We should have reached EOF once we're out of the while loop.
        self.close()

class NewTwitterCorpusReader(TwitterCorpusReader):
    
    CorpusView = GzipStreamBackedCorpusView
    
    def __init__(
        self, root, fileids=None, word_tokenizer=TweetTokenizer(), encoding='utf8'
    ):
        TwitterCorpusReader.__init__(self, root, fileids)
    
    def docs(self, fileids = None):
        #Concat is a method to create a ConcattenatedStreamBackedCorpusView
        return concat(
            [
                self.CorpusView(path, block_reader = self._read_tweets, encoding=enc)
                for (path, enc, fileid) in self.abspaths(fileids, True, True)
            ]
        )
    
    
    

In [61]:
corpus = NewTwitterCorpusReader('Twitter-Data/2019-11-07', ['Manhattan-2019-11-07-000.json.gz'])

In [62]:
len(corpus.docs())

153871498


40706

#### Solved - The issue has to do with the following process:
1. When TwitterCorpusReader uses the "docs" method, it relies on the abspaths function of the parent CorpusReader class, which initializes PathPointer Objects, not taking into consideration whether a specific file is zipped
2. This is because the TwitterCorpusReader is not expecting an individually zipped file but rather is expecting either unzipped files, or zipped by the root
3. This makes sense, as it is not relying on a CategorizedCorpusReader, which would differentiate based on the Root Object - this is important to take into consideration
4. Step 4 is to take into consideration CategorizedCorpusReader objects

In [111]:
from six import string_types, text_type

from nltk.tokenize import wordpunct_tokenize
from nltk.internals import slice_bounds
from nltk.data import PathPointer, FileSystemPathPointer, ZipFilePathPointer, GzipFileSystemPathPointer
from nltk.data import SeekableUnicodeStreamReader
from nltk.util import AbstractLazySequence, LazySubsequence, LazyConcatenation, py25
import struct
import io
import re

class GzipStreamBackedCorpusView(StreamBackedCorpusView):
    
    def __init__(self, fileid, block_reader = None, startpos = 0, encoding = 'utf8'):
        StreamBackedCorpusView.__init__(self, fileid, block_reader=block_reader, startpos=0, encoding='utf8')
        
        try:
            if isinstance(self._fileid, PathPointer):
                
                self._eofpos = self.getuncompressedsize(self._fileid)
            else:
                self._eofpos = os.stat(self._fileid).st_size
        except Exception as exc:
            raise ValueError('Unable to open or access %r -- %s' % (fileid, exc))

    def _open(self):
        if isinstance(self._fileid, PathPointer):
#             self._stream = self._fileid.open(self._encoding)
            self._stream = gzip.open(str(self._fileid), 'rb')
#             print(self._stream)
        elif self._encoding:
            print("here1")
            self._stream = SeekableUnicodeStreamReader(
                gzip.open(self._fileid, 'rb'), self._encoding
            )
        else:
            print("here2")
            self._stream = open(self._fileid, 'rb')
    
    def getuncompressedsize(self, filename):
        with gzip.GzipFile(filename, 'r') as fin:
            fin.seek(-4,2)
#             val = struct.unpack('I', fin.read(4))[0]
            val = fin.seek(0, io.SEEK_END)
            print(val)
            return val
    
    def iterate_from(self, start_tok):
        # Start by feeding from the cache, if possible.
        if self._cache[0] <= start_tok < self._cache[1]:
            for tok in self._cache[2][start_tok - self._cache[0] :]:
                yield tok
                start_tok += 1

        # Decide where in the file we should start.  If `start` is in
        # our mapping, then we can jump straight to the correct block;
        # otherwise, start at the last block we've processed.
        if start_tok < self._toknum[-1]:
            block_index = bisect.bisect_right(self._toknum, start_tok) - 1
            toknum = self._toknum[block_index]
            filepos = self._filepos[block_index]
        else:
            block_index = len(self._toknum) - 1
            toknum = self._toknum[-1]
            filepos = self._filepos[-1]

        # Open the stream, if it's not open already.
        if self._stream is None:
            self._open()

        # If the file is empty, the while loop will never run.
        # This *seems* to be all the state we need to set:
        if self._eofpos == 0:
            self._len = 0

        # Each iteration through this loop, we read a single block
        # from the stream.
        while filepos < self._eofpos:
            # Read the next block.
            self._stream.seek(filepos)
            self._current_toknum = toknum
            self._current_blocknum = block_index
            tokens = self.read_block(self._stream)
            assert isinstance(tokens, (tuple, list, AbstractLazySequence)), (
                'block reader %s() should return list or tuple.'
                % self.read_block.__name__
            )
            num_toks = len(tokens)
            new_filepos = self._stream.tell()
            if new_filepos <= filepos:
                print(new_filepos, filepos)
            assert new_filepos > filepos, (
                'block reader %s() should consume at least 1 byte (filepos=%d)'
                % (self.read_block.__name__, filepos)
            )

            # Update our cache.
            self._cache = (toknum, toknum + num_toks, list(tokens))

            # Update our mapping.
            assert toknum <= self._toknum[-1]
            if num_toks > 0:
                block_index += 1
                if toknum == self._toknum[-1]:
                    assert new_filepos > self._filepos[-1]  # monotonic!
                    self._filepos.append(new_filepos)
                    self._toknum.append(toknum + num_toks)
                else:
                    # Check for consistency:
                    assert (
                        new_filepos == self._filepos[block_index]
                    ), 'inconsistent block reader (num chars read)'
                    assert (
                        toknum + num_toks == self._toknum[block_index]
                    ), 'inconsistent block reader (num tokens returned)'

            # If we reached the end of the file, then update self._len
            if new_filepos == self._eofpos:
                self._len = toknum + num_toks
            # Generate the tokens in this block (but skip any tokens
            # before start_tok).  Note that between yields, our state
            # may be modified.
            for tok in tokens[max(0, start_tok - toknum) :]:
                yield tok
            # If we're at the end of the file, then we're done.
            assert new_filepos <= self._eofpos
            if new_filepos == self._eofpos:
                break
            # Update our indices
            toknum += num_toks
            filepos = new_filepos

        # If we reach this point, then we should know our length.
        assert self._len is not None
        # Enforce closing of stream once we reached end of file
        # We should have reached EOF once we're out of the while loop.
        self.close()

class NewTwitterCorpusReader(TwitterCorpusReader):
    
#     CorpusView = StreamBackedCorpusView
    
    def __init__(
        self, root, fileids=None, word_tokenizer=TweetTokenizer(), encoding='utf8'
    ):
        CategorizedCorpusReader.__init__(self, root, )
        TwitterCorpusReader.__init__(self, root, fileids)
        
        if isinstance(root, string_types) and not isinstance(root, PathPointer):
            m = re.match('(.*\.gz)/?(.*)$|', root) #'(.*\.zip)/?(.*\.gz)/?(.*)$|'
            zipfile, zipentry = m.groups()
            print(zipfile, zipentry)
            if zipfile:
                root = ZipFilePathPointer(zipfile, zipentry)
            else:
                root = FileSystemPathPointer(root)
                print(type(root))
        elif not isinstance(root, PathPointer):
            raise TypeError('CorpusReader: expected a string or a PathPointer')
        
        self._root = root
    
    def docs(self, fileids = None):
        #Concat is a method to create a ConcattenatedStreamBackedCorpusView
        return concat(
            [
                self.CorpusView(path, block_reader = self._read_tweets, encoding=enc)
                for (path, enc, fileid) in self.abspaths(fileids, True, True)
            ]
        )
    
    def abspaths(self, fileids=None, include_encoding=False, include_fileid=False):
        if fileids is None:
            fileids = self._fileids
        elif isinstance(fileids, string_types):
            fileids = [fileids]

        paths = [GzipFileSystemPathPointer(self._root.join(f)) for f in fileids]

        if include_encoding and include_fileid:
            return list(zip(paths, [self.encoding(f) for f in fileids], fileids))
        elif include_fileid:
            return list(zip(paths, fileids))
        elif include_encoding:
            return list(zip(paths, [self.encoding(f) for f in fileids]))
        else:
            return paths

        

In [112]:
corpus = NewTwitterCorpusReader('Twitter-Data/', ['Manhattan-2019-11-07-000.json.gz'])

None None
<class 'nltk.data.FileSystemPathPointer'>


In [114]:
#At the moment, it is also possible to pass it a root directory that is itself zipped
#As in the root directory is a sipped folder (holding the zipped files)
#Step 2:
# We need to work out a way to... integrate the current state of our dataset to the actual
# already built pipeline (which is built for html documents, and needs redesigning to additionally
# take into consideration location etc. and extract fulltext)

In [151]:
ROOT = r'Twitter-Data/Data/'
DOC_PATTERN = r'Manhattan.*\.json\.gz$'
PKL_PATTERN = r'^Manhattan\.pickle'

In [152]:
x = re.match(DOC_PATTERN,'Manhattan-2019-11-07-000.json.gz')

In [155]:
x[0]

'Manhattan-2019-11-07-000.json.gz'